# Problem Set 3, due FRIDAY, JUNE 5, 2020 @ 11:59pm

This problem set covers logistic regression, cross validation, and kernels.

*Please*:
1. Complete the information below with your name, e-mail address, and student ID number
2. Complete the assignment carefully, working through all parts
3. Before submitting, select "Restart Kernel and Run All Cells..." from the "Kernel" menu, to make sure this notebook runs cleanly.
4. Submit using the "submitps" command

<b>Your information</b>:<br>
Name: Hyuna Kwon <br>
e-mail address: hkwon019@ucr.edu <br>
SID #: 862063261 <br>

By submitting this notebook, you are asserting that the work presented is your own, was completed without external aid, and was completed for this offering of this course.

In this problem set, we will try bagging and boosting with decision trees as the base classifiers.

For all plots:
- Plot the error rate (vertical axis) versus the number of trees (horizontal axis)
- plot all data on a semi-log plot (# of trees being in log)
- try to avoid plotting for # of trees=2, as this is a weird case (hard to vote with just 2 base classifiers)
- Draw **testing** error with solid lines
- Draw **training** error with dashed lines
- Draw errors for the same method with the same color (differentiating by solid/dashed, as above)
- Use the same horizontal and vertical axis limits for **all** plots
- label axes and title your plots
- use a legend to distinguish the different lines on the plot

For this problem set, we will treat the "testing" data as cross-validation data.  That is, we will assume it is available to us to judge the future performance on true testing data.

First, load the data, as before:

In [54]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from IPython import display
import time

trainX = np.loadtxt('spamtrainX.data')
trainY = np.loadtxt('spamtrainY.data')[:,np.newaxis]
mux = np.mean(trainX,axis=0)
trainX -= mux
stdx = np.std(trainX,axis=0)
trainX /= stdx

testX = np.loadtxt('spamtestX.data')
testX = (testX-mux)/stdx
testY = np.loadtxt('spamtestY.data')[:,np.newaxis]

## Question 1 (4 points)

***
First, let's consider the VC-dimension of decision trees.

### part (a) (2 points)

What is the VC-dimension of a decision trees of depth $d$ when used on one-dimensional data ($n=1$)?

Prove your answer.

Answer : VC-dimension of a decision trees of a depth $d$ when used on one-dimensional data is 2$^{d-1}$([log$_2$(3-d)]+1) <br>
Proof :

If d = 1 $\rightarrow$ VC-dimension is 2. <br>
Because $x_1 \leq t_1$ can only shatter one or two data.

If d = k+1 $\rightarrow$ VC-dimension is

VCDimension lower-bound-binary(DT) <br>
if DT is a leaf node <br>
    $\quad$return 1 <br>
if left and right subtress of DT are leaves <br>
    $\quad$return 2 <br>
DT$_L$ = Left subtree of DT <br>
DT$_R$ = Right subtree of DT <br>
return lower-bound-binary(DT$_L$) + lower-bound-binary(DT$_R$) <br>


### part (b) (2 points)

Is this the same or different than the VC-dimension of a decision trees of depth $d$ when used on two-dimensional data ($n=2$)?

Clearly explain why.

Answer: This is different from the VC-dimension of a decision trees of depth $d$ when used on two-dimensional data.

When d = 2 on one-dimentional data, VC-dimension = 2
When d = 2 on two-dimensional data, VC-dimension = 4

## Question 2 (11 points)

***

### Supplied learning functions:

Below are three functions that will train a decision tree, and predict using that decision tree.
They use sklearn.  **You may use these supplied functions, but not sklearn for this problem set.**

In [217]:
# returns a decision tree to be used with treepred, below
# samplewts can be added to provide a different weight per X row
# (b/c this will be used with an ensemble, no pruning is done!)
def traindt(X,Y,maxdepth,samplewts=None):
    from sklearn.tree import DecisionTreeClassifier # do not import anything from sklearn other than this line inside this function
    tree = DecisionTreeClassifier(criterion='gini',splitter='best',max_depth=maxdepth)
    tree.fit(X,Y,sample_weight=samplewts)
    return tree

# will return array m-by-2
# each row is a different row from x
# each column is a different class
# value is prob of that class for that example
def treepred(tree,x):
    return tree.predict_proba(x)

# same as treepred, but returns the difference between the
# probabilities, which can be used as a discriminator (>0 => +1, <0 => -1)
def treeout(tree,x):
    f = treepred(tree,x)
    return (f[:,1]-f[:,0])[:,np.newaxis]

### part (a) (7 points) Bagging (2 points) and Boosting (3 points)

Below, write your code to learn via bagging and boosting (two separate methods).  Your functions should take in the number of trees (rounds of bagging/boosting) and the maximum depth of the tree to be used on each round.

In [289]:
#maxdepth = 3
#tree = traindt(trainX,trainY,maxdepth)


def bagging(data,X,Y,depth,numberOfTrees):
    bootstrapData = bootstrap(data)
    train_x = X
    train_y = Y
    trees = traindt(train_x,train_y,depth)
    return trees

def bootstrap(data):
    # The following method generates a bootstrap dataset using the input dataset
    # It picks random incides from input data (without replacement)
    # and uses those indices to create the bootstrap dataset of same size
    
    [m,d] = data.shape
    indices = np.random.randint(m-1,size = m)
    indices = indices.T + 1
    bootstrapData = data[indices,:]
    return bootstrapData

def boosting(data,X,Y,depth,numberOfTrees):
    
    [m,d] = data.shape
    print(m,d)
    alpha = np.ones(m)
    trees = np.zeros((1,numberOfTrees))
    w = np.zeros(numberOfTrees)
    for i in range(numberOfTrees):
        train_x = X
        train_y = Y
        trees = traindt(train_x,train_y,depth,alpha)
        y_pred = treeout(trees,train_x)
        indices = np.argwhere(np.sign(y_pred)!=np.sign(train_y))
        error = (sum(alpha[indices])/sum(alpha))
        error = error[1]
        w[i] = np.log((1-error)/error)
        alpha[indices] = alpha[indices]*np.exp(w[i])
    
    return [trees,w]


def predictBagging(x,trees,numberOfTrees):
#This method uses the trees generated by the bagging function to
# predict the output for input data. It uses nubmerOfTrees param to use
# only a part of trees to predict the output.
    y = np.zeros(x.shape[0])
    
    for i in range(numberOfTrees):
        y = y + treeout(trees[i],x)
        
    y = y/numberOfTrees
    y = np.where(y >= 0, 1, -1)
    
    return y
    
def predictBoosting(x,trees,w,numberOfTrees):
    
    y = np.zeros(x.shape[0])
    for i in range(numberOfTrees):
        y = y + w[i]*treepred(trees,x)
    
    y = np.where(y >= 0, 1, -1)
    
    return y



### part (b) (3 points) Plotting Bagging and Boosting

Below plot the error rates (see top of the assignment) in two plots:  1 for bagging, 1 for boosting.  Plot each for 1 to 10000 rounds (plotted points should be distributed evenly on a log-scale).  Plot each for trees of depth 1, 2, and 4.  

[My solutions take about 5 minutes per plot.  You may want to test with 1 to 100 or 1 to 1000 rounds first.]

In [293]:
    print(trainX.shape)
    print(trainY.shape)
       
    numberOfTreesArray = np.floor(np.logspace(1,3,10)).astype(int)
    print(numberOfTreesArray)
    depths = np.array([1,2,3])
    
    trainData = np.hstack((trainX,trainY))
    
    for depth in depths:
        index = 1
        baggingTrainErrorArray = np.zeros((numberOfTreesArray.size))
        baggingTestErrorArray = np.zeros((numberOfTreesArray.size))
        boostingTrainErrorArray = np.zeros((numberOfTreesArray.size))
        boostingTestErrorArray = np.zeros((numberOfTreesArray.size))
        
        treesBagging = np.zeros(numberOfTrees)
        treesBoosting = np.zeros(numberOfTrees)
        w = np.zeros(numberOfTrees)
              
        #treesBagging = bagging(trainData,trainX,trainY,depth,numberOfTreesArray[-1])
        #treesBoosting,w = boosting(trainData,trainX,trainY,depth,numberOfTreesArray[-1])
        
        for numberOfTrees in numberOfTreesArray:
            x_train = trainX
            y_train = trainY
            
            y_pred = np.zeros(trainX.shape[0])
            
            for i in range(numberOfTrees):
                trees = bagging(trainData,trainX,trainY,depth,numberOfTrees)
                y_pred = y_pred + treeout(trees,trainX)
        
            y_pred = y_pred/numberOfTrees
            y_pred = np.where(y_pred >= 0, 1, -1)
            print(y_pred)
            
            mismatches = np.argwhere(y_train*y_pred<0)
            print(mismatches)
            baggingTrainErrorArray[index] = (mismatches.shape[0]/y_train.shape[0])*100
            
            print(baggingTrainErrorArray[index])
            
            
            y_pred = predictBoosting(x_train,treesBoosting,w,numberOfTrees)
            mismatches = np.argwhere(sign(y_train)!=sign(y_pred))
            boostingTrainErrorArray[index] = (length(mismatches)/length(y_train))*100
            
            x_test = testX
            y_test = testY
            
            print(treesBagging)
            y_pred = predictBagging(x_test,treesBagging,numberOfTrees)
            mismatches = np.argwhere(np.sign(y_test)!=np.sign(y_pred))
            baggingTestErrorArray[index] = (length(mismatches)/length(y_test))*100
            
            y_pred = predictBoosting(x_test,treesBoosting,w,numberOfTrees)
            mismatches = np.argwhere(sign(y_test)!=sign(y_pred))
            boostingTrainErrorArray[index] = (length(mismatches)/length(y_test))*100
            
            index = index + 1
        
        plt.semilogx(numberOfTreesArray,baggingTrainErrorArray)
        plt.semilogx(numberOfTreesArray,baggingTestErrorArray)
        plt.semilogx(numberOfTreesArray,boostingTrainErrorArray)
        plt.semilogx(numberOfTreesArray,boostingTestErrorArray)
        plt.show()

(3000, 57)
(3000, 1)
[  10   16   27   46   77  129  215  359  599 1000]
[[-1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ...  1  1  1]
 [ 1  1  1 ...  1  1  1]
 ...
 [-1 -1 -1 ... -1 -1 -1]
 [ 1  1  1 ...  1  1  1]
 [-1 -1 -1 ... -1 -1 -1]]
[[   1    0]
 [   1    1]
 [   1    2]
 ...
 [2996 2997]
 [2996 2998]
 [2996 2999]]
62800.0


AttributeError: 'numpy.ndarray' object has no attribute 'predict_proba'

### part (c) (3 points) So what?

What are the differences between the bagging and boosting plots and between the training and testing plots?  What do these differences say about
(1) the choice of boosting versus bagging?  (2) the choice of the base classifier?  (3) the number of rounds to use?  (4) overfitting for these methods?

(1) There's not an outright winner; it depends on the data, the simulation and the circumstances. If the problem is that the single model gets a very low performance, Bagging will rarely get a better bias. However, Boosting could generate a combined model with lower errors as it optimizes the advantages and reduces pitfalls of the single model.

(2) Bagging is an ensemble generation method that uses variations of samples used to train base classifiers. For each classifier to be generated, Bagging selects (with repetition) N samples from the training set with size N and train a base classifier. Boosting generates an ensemble by adding classifiers that correctly classify difficult samples.For each iteration, boosting updates the weights of the samples, so that, samples that are isclassified by the ensemble can have a higher weight, and therefore, higher probability of being selected for training the new classifier.

(3)

(4) If the difficulty of the single model is over-fitting, then Bagging is the best option. Boosting for its part doesn't help to avoid over-fitting.

In Bagging plots, we can clearly see that as we increase the depth the quality of classifier improves. Also, as we increase the number of trees the quality of classifier goes up. This is due to the fact that, where some classifier may fail to predict the correct value at a point, others might do well which improves the overall quality.

Similarly in Boosting plots, increasing depth or number of trees increases the quality of the classifier. But in this case the quality of predictions is even better than the first one. This is due to the fact that individual data points have weight associated with them which helps in better prediction for individual points.

Plots explains the effect of bagging and boosting on bias and variance. It is evident from the plots that bagging improves the variance whereas boosting improves bias of the underlying classifier. Detail explaination has been provided below (in the last part of  the question).

Bias and Variance:
In bagging we essentially take an average of all the classifiers generated using the bootstrap data in order to predict the output. Thus bagging doesnt affect the bias (in theory) but reduces the variance. Whereas boosting decreases the bias but increase variance which can be seen in the plots for part a and part b.

In the first plot, the difference between the training error and the testing error for fixed number of trees is low which suggests that bagging improves the variance of the underlying classifier. Also, as we increase the number of trees the error rate goes down (immproves quality of prediction).

In the second plot, the difference between the training error and the testing error for fixed number of trees is relatively high which suggests that boosting increases the variance. But As we increase the number of trees, the error rate quickly goes down to zero which suggest that boosting improves the bias of the underlying classifier. Also, as we increase the number of trees the error rate goes down (immproves quality of prediction).